In [1560]:
import numpy as np
import pandas as pd

DIR = "/Users/rithikpothuganti/quantsc/Sports-Betting"
START_YEAR = 2002
END_YEAR = 2023
PRED_YEAR = 2023


In [1561]:
stat_categories = ['TEAM', 'W', 'L', 'PW', 'PL', 'MOV', 'SOS', 'SRS', 'ORtg', 
 'DRtg', 'NRtg', 'Pace', 'FTr', '3PAr', 'TS%', 'TOV%', 'ORB%', 
 'FT/FGA', 'eFG%', 'TOV%', 'DRB%', 'FT/FGA', 'Attend./G']

"""
['TEAM', 'Age', 'W', 'L', 'PW', 'PL', 'MOV', 'SOS', 'SRS', 'ORtg', 
 'DRtg', 'NRtg', 'Pace', 'FTr', '3PAr', 'TS%', 'eFG%', 'TOV%', 'ORB%', 
 'FT/FGA', 'eFG%', 'TOV%', 'DRB%', 'FT/FGA', 'Attend./G', 'G', 'MP', 'FG', 
 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'FT', 'FTA', 'FT%', 
 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'opp_G', 
 'opp_MP', 'opp_FG', 'opp_FGA', 'opp_FG%', 'opp_3P', 'opp_3PA', 'opp_3P%', 
 'opp_2P', 'opp_2PA', 'opp_2P%', 'opp_FT', 'opp_FTA', 'opp_FT%', 'opp_ORB', 
 'opp_DRB', 'opp_TRB', 'opp_AST', 'opp_STL', 'opp_BLK', 'opp_TOV', 'opp_PF', 'opp_PTS', 'PLAYOFF_WINS']
"""

"\n['TEAM', 'Age', 'W', 'L', 'PW', 'PL', 'MOV', 'SOS', 'SRS', 'ORtg', \n 'DRtg', 'NRtg', 'Pace', 'FTr', '3PAr', 'TS%', 'eFG%', 'TOV%', 'ORB%', \n 'FT/FGA', 'eFG%', 'TOV%', 'DRB%', 'FT/FGA', 'Attend./G', 'G', 'MP', 'FG', \n 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'FT', 'FTA', 'FT%', \n 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'opp_G', \n 'opp_MP', 'opp_FG', 'opp_FGA', 'opp_FG%', 'opp_3P', 'opp_3PA', 'opp_3P%', \n 'opp_2P', 'opp_2PA', 'opp_2P%', 'opp_FT', 'opp_FTA', 'opp_FT%', 'opp_ORB', \n 'opp_DRB', 'opp_TRB', 'opp_AST', 'opp_STL', 'opp_BLK', 'opp_TOV', 'opp_PF', 'opp_PTS', 'PLAYOFF_WINS']\n"

In [1562]:
team_stats = pd.read_csv(f"{DIR}/data/NBA_team_stats/{START_YEAR}/team_stats.csv", usecols=lambda column: column.startswith('Unnamed') == False)
playoff_wins = pd.read_csv(f"{DIR}/data/NBA_playoff_wins/{START_YEAR}/playoff_wins.csv", usecols=lambda column: column.startswith('Unnamed') == False)
for year in range(START_YEAR+1, END_YEAR):
    playoff_wins_curr = pd.read_csv(f"{DIR}/data/NBA_playoff_wins/{year}/playoff_wins.csv", usecols=lambda column: column.startswith('Unnamed') == False)
    playoff_wins = pd.concat([playoff_wins, playoff_wins_curr])
    team_stats_curr = pd.read_csv(f"{DIR}/data/NBA_team_stats/{year}/team_stats.csv", usecols=lambda column: column.startswith('Unnamed') == False)
    team_stats = pd.concat([team_stats, team_stats_curr])

data = pd.merge(team_stats, playoff_wins, how='left').fillna(-1)
# no_playoff_wins_df = data[data['PLAYOFF_WINS'] == -1]
# data = data.drop(index=no_playoff_wins_df.index)
# data = pd.merge(team_stats, playoff_wins).fillna(0)
data = data[stat_categories + ['PLAYOFF_WINS']]
data


,TEAM,W,L,PW,PL,MOV,SOS,SRS,ORtg,DRtg,...,TS%,TOV%,ORB%,FT/FGA,eFG%,TOV%,DRB%,FT/FGA,Attend./G,PLAYOFF_WINS
0,SAC_2002,61.0,21.0,61,21,7.61,0.00,7.61,109.0,101.1,...,0.539,12.4,27.9,0.231,0.497,12.4,71.7,0.231,17317.0,10.0
1,LAL_2002,58.0,24.0,60,22,7.12,0.03,7.15,109.4,101.7,...,0.534,11.8,28.8,0.218,0.498,11.8,72.0,0.218,18995.0,15.0
2,SAS_2002,58.0,24.0,59,23,6.21,0.07,6.28,106.5,99.7,...,0.539,13.8,27.2,0.262,0.492,13.8,71.9,0.262,22107.0,4.0
3,NJN_2002,52.0,30.0,53,29,4.16,-0.49,3.67,104.0,99.5,...,0.515,13.4,29.2,0.206,0.476,13.4,71.6,0.206,13761.0,11.0
4,DAL_2002,57.0,25.0,53,29,4.26,0.16,4.41,112.2,107.7,...,0.553,11.3,26.1,0.232,0.507,11.3,70.7,0.232,19580.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
622,DET_2022,23.0,59.0,22,60,-7.72,0.37,-7.36,106.0,113.8,...,0.533,12.6,23.4,0.194,0.494,12.6,75.6,0.194,16184.0,-1.0
623,ORL_2022,22.0,60.0,21,61,-8.00,0.33,-7.67,104.5,112.5,...,0.538,13.0,19.8,0.175,0.503,13.0,77.2,0.175,15192.0,-1.0
624,OKC_2022,24.0,58.0,21,61,-8.10,0.20,-7.90,104.6,112.8,...,0.530,12.5,21.8,0.169,0.497,12.5,76.1,0.169,14515.0,-1.0
625,HOU_2022,20.0,62.0,21,61,-8.48,0.22,-8.26,108.4,116.7,...,0.565,14.5,21.7,0.202,0.534,14.5,74.4,0.202,15585.0,-1.0


In [1563]:
playoff_teams_2023 = pd.read_csv(f"{DIR}/data/NBA_playoff_wins/{PRED_YEAR}/playoff_wins.csv", usecols=lambda column: column.startswith('Unnamed') == False)
playoff_teams_2023 = playoff_teams_2023['TEAM'].to_numpy()
print(playoff_teams_2023)

stats_2023 = pd.read_csv(f"{DIR}/data/NBA_team_stats/{PRED_YEAR}/team_stats.csv", usecols=lambda column: column.startswith('Unnamed') == False)
stats_2023 = stats_2023[stats_2023['TEAM'].isin(playoff_teams_2023)]
stats_2023.reset_index(drop=True, inplace=True)
stats_2023 = stats_2023[stat_categories]

stats_2023


['PHI_2023' 'DEN_2023' 'PHO_2023' 'BOS_2023' 'MIA_2023' 'LAL_2023'
 'NYK_2023' 'GSW_2023' 'SAC_2023' 'CLE_2023' 'MEM_2023' 'MIL_2023'
 'ATL_2023' 'LAC_2023' 'MIN_2023' 'BRK_2023']


,TEAM,W,L,PW,PL,MOV,SOS,SRS,ORtg,DRtg,...,3PAr,TS%,TOV%,ORB%,FT/FGA,eFG%,TOV%,DRB%,FT/FGA,Attend./G
0,BOS_2023,57.0,25.0,57,25,6.52,-0.15,6.38,118.0,111.5,...,0.480,0.600,12.0,22.1,0.197,0.566,12.0,78.5,0.197,18689
1,CLE_2023,51.0,31.0,55,27,5.38,-0.15,5.23,116.1,110.6,...,0.371,0.590,12.3,23.6,0.206,0.556,12.3,76.3,0.206,18958
2,PHI_2023,54.0,28.0,52,30,4.32,0.06,4.37,117.7,113.3,...,0.389,0.608,12.6,21.6,0.250,0.563,12.6,77.2,0.250,20470
3,MEM_2023,51.0,31.0,51,31,3.94,-0.34,3.60,115.1,111.2,...,0.372,0.570,11.7,26.5,0.190,0.540,11.7,75.9,0.190,17264
4,MIL_2023,58.0,24.0,50,32,3.63,-0.02,3.61,115.4,111.9,...,0.446,0.583,12.7,25.0,0.184,0.555,12.7,77.8,0.184,17531
5,DEN_2023,53.0,29.0,49,33,3.33,-0.29,3.04,117.6,114.2,...,0.361,0.601,13.1,24.8,0.194,0.573,13.1,76.4,0.194,19235
6,NYK_2023,47.0,35.0,48,34,2.93,0.06,2.99,117.8,114.8,...,0.400,0.577,11.4,28.3,0.217,0.541,11.4,77.1,0.217,19393
7,SAC_2023,48.0,34.0,47,35,2.65,-0.35,2.30,119.4,116.8,...,0.423,0.608,12.0,22.7,0.225,0.572,12.0,77.2,0.225,17451
8,PHO_2023,45.0,37.0,46,36,2.07,0.01,2.08,115.1,113.0,...,0.362,0.570,12.0,26.6,0.191,0.535,12.0,76.0,0.191,17071
9,GSW_2023,44.0,38.0,45,37,1.80,-0.15,1.66,116.1,114.4,...,0.479,0.600,14.1,24.4,0.178,0.571,14.1,76.0,0.178,18064


In [1564]:
# Linear Regression

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [1565]:
# Define the features and the target
X = data.drop(columns=['TEAM', 'PLAYOFF_WINS'])
# for col in X.columns:
#     col_mean = X[col].mean()
#     col_min = X[col].min()
#     col_max = X[col].max()
#     X[col] = (X[col] - col_mean) / (col_max - col_min)
#     X[col] = X[col] * 2 - 1
y = data['PLAYOFF_WINS']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

X = X.fillna(0)


In [1566]:
# Create and fit the model
model = LinearRegression()
model.fit(X_train, y_train)


LinearRegression()

In [1567]:
# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model using mean squared error and R-squared
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error: ", mse)
print("R-squared: ", r2)

Mean Squared Error:  9.085159006306538
R-squared:  0.5642799138923702


In [1568]:
# Neural Network
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F


In [1569]:
class NBAPlayoffPredictor(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NBAPlayoffPredictor, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.bn1 = nn.BatchNorm1d(hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.bn2 = nn.BatchNorm1d(hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.bn3 = nn.BatchNorm1d(hidden_size)
        self.fc4 = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = F.relu(self.bn3(self.fc3(x)))
        x = self.dropout(x)
        x = self.fc4(x)
        return x


In [1570]:
# Define the features and the target
X = data.drop(columns=['TEAM', 'PLAYOFF_WINS']).values
y = data['PLAYOFF_WINS'].values
stats_2023_values = stats_2023.drop(columns=['TEAM']).values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
stats_2023_values = scaler.transform(stats_2023_values)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)
stats_2023_tensor = torch.tensor(stats_2023_values, dtype=torch.float32).to('cpu')

# Create DataLoader objects
train_data = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)

test_data = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)


In [1571]:
# Initialize the model, loss function, and optimizer
input_size = X_train.shape[1]
hidden_size = 64
output_size = 1
model = NBAPlayoffPredictor(input_size, hidden_size, output_size)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Train the model
patience = 50
early_stop_counter = 0
best_val_loss = float('inf')
epochs = 1000

for epoch in range(epochs):
    # Training loop
    model.train()
    for inputs, targets in train_loader:
        inputs, targets = inputs.to('cpu'), targets.to('cpu')
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

    # Validation loop
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to('cpu'), targets.to('cpu')
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item()

    val_loss /= len(test_loader)
    print(f"Epoch {epoch+1}, Validation Loss: {val_loss:.4f}")

    # Early stopping logic
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        early_stop_counter = 0
    else:
        early_stop_counter += 1

    if early_stop_counter >= patience:
        print(f"Early stopping triggered. Stopping training at epoch {epoch+1}.")
        break

Epoch 1, Validation Loss: 28.5779
Epoch 2, Validation Loss: 28.2312
Epoch 3, Validation Loss: 27.8190
Epoch 4, Validation Loss: 27.6032
Epoch 5, Validation Loss: 27.1720
Epoch 6, Validation Loss: 26.9944
Epoch 7, Validation Loss: 26.5689
Epoch 8, Validation Loss: 26.2776
Epoch 9, Validation Loss: 25.8433
Epoch 10, Validation Loss: 25.5721
Epoch 11, Validation Loss: 25.3342
Epoch 12, Validation Loss: 25.1065
Epoch 13, Validation Loss: 24.7073
Epoch 14, Validation Loss: 24.3692
Epoch 15, Validation Loss: 23.9544
Epoch 16, Validation Loss: 23.2559
Epoch 17, Validation Loss: 22.8743
Epoch 18, Validation Loss: 22.7912
Epoch 19, Validation Loss: 22.5066
Epoch 20, Validation Loss: 21.8087
Epoch 21, Validation Loss: 22.0425
Epoch 22, Validation Loss: 21.5751
Epoch 23, Validation Loss: 21.2559
Epoch 24, Validation Loss: 20.9730
Epoch 25, Validation Loss: 20.0519
Epoch 26, Validation Loss: 20.2579
Epoch 27, Validation Loss: 19.6007
Epoch 28, Validation Loss: 19.0336
Epoch 29, Validation Loss: 18

In [1572]:
model.eval()
with torch.no_grad():
    test_loss = 0


In [1573]:

model.eval()

# Initialize a variable to store the test loss
test_loss = 0

# No need to compute gradients during evaluation, so use torch.no_grad()
with torch.no_grad():
    for inputs, targets in test_loader:
        # Move inputs and targets to the appropriate device (CPU or GPU)
        inputs, targets = inputs.to('cpu'), targets.to('cpu')

        # Forward pass
        outputs = model(inputs)

        # Compute the loss
        loss = criterion(outputs, targets)

        # Accumulate the test loss
        test_loss += loss.item()

# Compute the average test loss
average_test_loss = test_loss / len(test_loader)

# Print the average test loss
print("Test Loss: {:.4f}".format(average_test_loss))


Test Loss: 6.4432


In [1574]:
def r_squared(y_true, y_pred):
    ssr = torch.sum((y_true - y_pred)**2)
    sst = torch.sum((y_true - torch.mean(y_true))**2)
    r2 = 1 - (ssr / sst)
    return r2.item()

# Assuming the following variables are defined:
# val_loader: DataLoader for your validation or test dataset
# model: your trained neural network model
# device: the device to use (e.g., 'cpu' or 'cuda')

model.eval()
y_true = []
y_pred = []

with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to('cpu'), targets.to('cpu')
        outputs = model(inputs)
        y_true.append(targets)
        y_pred.append(outputs)

y_true = torch.cat(y_true).detach()
y_pred = torch.cat(y_pred).detach()

r2 = r_squared(y_true, y_pred)
print(f"Neural Network R-squared value: {r2:.4f}")


Neural Network R-squared value: 0.7158


In [1575]:
# Neural network predictions

with torch.no_grad():
    predictions = model(stats_2023_tensor)

predictions_np = predictions.cpu().numpy()

for team, pred in zip(stats_2023["TEAM"].to_numpy(), predictions_np):
    print(f"{team}: {pred[0]}")

BOS_2023: 8.126073837280273
CLE_2023: 6.227416038513184
PHI_2023: 6.008488178253174
MEM_2023: 4.53537654876709
MIL_2023: 6.921087265014648
DEN_2023: 6.373075485229492
NYK_2023: 2.3562073707580566
SAC_2023: 4.613025188446045
PHO_2023: 1.6474496126174927
GSW_2023: 3.5326528549194336
BRK_2023: 2.2562508583068848
LAL_2023: 0.6708236932754517
LAC_2023: 2.851802110671997
ATL_2023: -0.3533651828765869
MIN_2023: 1.7461185455322266
MIA_2023: 0.7542967796325684


In [1576]:
# K Nearest Neighbors

from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor(n_neighbors=35)
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)

mse_knn = mean_squared_error(y_test, y_pred_knn)
r2_knn = r2_score(y_test, y_pred_knn)

print("KNN Mean Squared Error: ", mse_knn)
print("KNN R-squared: ", r2_knn)

KNN Mean Squared Error:  7.209931972789115
KNN R-squared:  0.6801835108463627


In [1577]:
# KNN Predictions

preds_2023 = knn.predict(stats_2023_values)

for team, prediction in zip(stats_2023["TEAM"].to_numpy(), preds_2023):
    print(f"{team}: {prediction:.2f} predicted playoff wins")

BOS_2023: 7.49 predicted playoff wins
CLE_2023: 6.29 predicted playoff wins
PHI_2023: 5.77 predicted playoff wins
MEM_2023: 4.86 predicted playoff wins
MIL_2023: 6.03 predicted playoff wins
DEN_2023: 6.31 predicted playoff wins
NYK_2023: 4.11 predicted playoff wins
SAC_2023: 5.26 predicted playoff wins
PHO_2023: 2.63 predicted playoff wins
GSW_2023: 4.14 predicted playoff wins
BRK_2023: 3.49 predicted playoff wins
LAL_2023: 2.20 predicted playoff wins
LAC_2023: 2.97 predicted playoff wins
ATL_2023: 1.86 predicted playoff wins
MIN_2023: 2.20 predicted playoff wins
MIA_2023: 2.26 predicted playoff wins


In [1578]:
# SVM 

from sklearn.svm import SVR

svm = SVR(kernel='rbf', C=1)
svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_test)

mse_svm = mean_squared_error(y_test, y_pred_svm)
r2_svm = r2_score(y_test, y_pred_svm)

print("SVM Mean Squared Error: ", mse_svm)
print("SVM R-squared: ", r2_svm)

SVM Mean Squared Error:  7.770752994279217
SVM R-squared:  0.6553067421315621


In [1579]:
# SVM Predictions

predictions = svm.predict(stats_2023_values)

for team, prediction in zip(stats_2023["TEAM"].to_numpy(), predictions):
    print(f"{team}: {prediction:.2f} playoff wins")

BOS_2023: 6.82 playoff wins
CLE_2023: 6.09 playoff wins
PHI_2023: 4.28 playoff wins
MEM_2023: 4.35 playoff wins
MIL_2023: 5.30 playoff wins
DEN_2023: 4.08 playoff wins
NYK_2023: 2.49 playoff wins
SAC_2023: 3.21 playoff wins
PHO_2023: 1.87 playoff wins
GSW_2023: 1.68 playoff wins
BRK_2023: 1.86 playoff wins
LAL_2023: 1.06 playoff wins
LAC_2023: 1.56 playoff wins
ATL_2023: 0.64 playoff wins
MIN_2023: 0.97 playoff wins
MIA_2023: 0.95 playoff wins


In [1580]:
# Random Forest Regressor

from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=100)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print("Random Forest Mean Squared Error: ", mse_rf)
print("Random Forest R-squared: ", r2_rf)

Random Forest Mean Squared Error:  6.0191261904761895
Random Forest R-squared:  0.7330049973736268


In [1581]:
# Random Forest Predictions
predictions = rf.predict(stats_2023_values)
for team, prediction in zip(stats_2023["TEAM"].to_numpy(), predictions):
    print(f"{team}: {prediction:.2f} predicted playoff wins")

BOS_2023: 11.70 predicted playoff wins
CLE_2023: 6.53 predicted playoff wins
PHI_2023: 8.23 predicted playoff wins
MEM_2023: 6.78 predicted playoff wins
MIL_2023: 8.48 predicted playoff wins
DEN_2023: 10.48 predicted playoff wins
NYK_2023: 1.90 predicted playoff wins
SAC_2023: 3.06 predicted playoff wins
PHO_2023: 0.78 predicted playoff wins
GSW_2023: 2.54 predicted playoff wins
BRK_2023: 2.43 predicted playoff wins
LAL_2023: 0.72 predicted playoff wins
LAC_2023: 2.22 predicted playoff wins
ATL_2023: 0.73 predicted playoff wins
MIN_2023: 1.73 predicted playoff wins
MIA_2023: 1.49 predicted playoff wins
